# A Comprehensive Guide to Reinforcement Learning

Welcome to this comprehensive guide to Reinforcement Learning (RL). This notebook will walk you through the fundamental concepts, algorithms, and practical implementations of RL. We will cover everything from the basics of agents and environments to advanced topics like Deep Q-Networks.

## 1. Introduction to Reinforcement Learning

Reinforcement Learning is a subfield of machine learning where an **agent** learns to make decisions by interacting with an **environment**. The agent's goal is to maximize the cumulative **reward** it receives over time.

The core components of an RL system are:

*   **Agent:** The learner or decision-maker.
*   **Environment:** The external world with which the agent interacts.
*   **State (s):** A representation of the environment's current situation.
*   **Action (a):** A decision made by the agent.
*   **Reward (r):** A scalar feedback signal from the environment that indicates how well the agent is doing.

The agent and environment interact in a sequence of discrete time steps. At each time step *t*, the agent receives a state *S_t* and selects an action *A_t*. The environment responds with a new state *S_{t+1}* and a reward *R_{t+1}*. The agent's objective is to learn a **policy** (a mapping from states to actions) that maximizes the total expected reward.

## 2. Markov Decision Processes (MDPs)

Most RL problems can be formalized as Markov Decision Processes (MDPs). An MDP is a mathematical framework for modeling decision-making in situations where outcomes are partly random and partly under the control of a decision-maker.

An MDP is defined by a tuple `(S, A, P, R, γ)`:

*   **S:** A finite set of states.
*   **A:** A finite set of actions.
*   **P:** The state transition probability function, `P(s' | s, a)`, which is the probability of transitioning to state `s'` from state `s` after taking action `a`.
*   **R:** The reward function, `R(s, a, s')`, which is the expected reward received after transitioning from state `s` to state `s'` by taking action `a`.
*   **γ (gamma):** The discount factor, `0 <= γ <= 1`, which trades off the importance of immediate versus future rewards.

## 3. The Bellman Equations

The Bellman equations are a set of equations that decompose the value function into the immediate reward plus the discounted future values. They are fundamental to many RL algorithms.

### State-Value Function (V(s))

The state-value function `V(s)` is the expected return when starting in state `s` and following a policy `π` thereafter.

The Bellman expectation equation for `V(s)` is:

`V_π(s) = E_π[R_{t+1} + γV_π(S_{t+1}) | S_t = s]`

### Action-Value Function (Q(s, a))

The action-value function `Q(s, a)` is the expected return when starting in state `s`, taking action `a`, and then following a policy `π`.

The Bellman expectation equation for `Q(s, a)` is:

`Q_π(s, a) = E_π[R_{t+1} + γQ_π(S_{t+1}, A_{t+1}) | S_t = s, A_t = a]`

## 4. Setting Up the Environment

Before we dive into the algorithms, let's install the necessary libraries. We will be using `gymnasium` for the environments and `numpy` for numerical computations. Later on, we will also use `stable-baselines3` for Deep RL.

In [ ]:
!pip install gymnasium numpy

## 5. A Simple GridWorld Environment

To understand the RL algorithms, we will use a simple GridWorld environment. Our GridWorld is a 4x4 grid. The agent's goal is to navigate from a starting position to a goal position.

Here are the details of our GridWorld:

*   **State Space:** A 4x4 grid, so there are 16 states.
*   **Action Space:** Four actions: Up, Down, Left, Right.
*   **Rewards:**
    *   +1 for reaching the goal.
    *   -1 for falling into a hole.
    *   0 for all other moves.
*   **Dynamics:** The agent moves one step in the chosen direction. If the agent hits a wall, it stays in the same place.

In [ ]:
import numpy as np
import sys

class GridWorldEnv:
    def __init__(self, grid_size=4, start_state=0):
        self.grid_size = grid_size
        self.n_states = grid_size * grid_size
        self.n_actions = 4  # 0: Left, 1: Down, 2: Right, 3: Up
        self.start_state = start_state
        self.agent_state = start_state

        self.goal_state = self.n_states - 1
        self.holes = [5, 7, 11, 12]

    def reset(self):
        self.agent_state = self.start_state
        return self.agent_state

    def step(self, action):
        row, col = self._state_to_coords(self.agent_state)

        if action == 0:  # Left
            col = max(0, col - 1)
        elif action == 1:  # Down
            row = min(self.grid_size - 1, row + 1)
        elif action == 2:  # Right
            col = min(self.grid_size - 1, col + 1)
        elif action == 3:  # Up
            row = max(0, row - 1)

        next_state = self._coords_to_state(row, col)
        self.agent_state = next_state

        if next_state == self.goal_state:
            reward = 1.0
            done = True
        elif next_state in self.holes:
            reward = -1.0
            done = True
        else:
            reward = 0.0
            done = False

        return next_state, reward, done, {}

    def _state_to_coords(self, state):
        return state // self.grid_size, state % self.grid_size

    def _coords_to_state(self, row, col):
        return row * self.grid_size + col

    def render(self):
        grid = np.full((self.grid_size, self.grid_size), '_', dtype=str)
        for hole in self.holes:
            row, col = self._state_to_coords(hole)
            grid[row, col] = 'H'
        goal_row, goal_col = self._state_to_coords(self.goal_state)
        grid[goal_row, goal_col] = 'G'
        agent_row, agent_col = self._state_to_coords(self.agent_state)
        grid[agent_row, agent_col] = 'A'
        print('\n'.join([' '.join(row) for row in grid]))

# Let's test the environment
env = GridWorldEnv()
state = env.reset()
env.render()

print("\nTaking action: Right")
next_state, reward, done, _ = env.step(2) # 2 is Right
env.render()
print(f"Next State: {next_state}, Reward: {reward}, Done: {done}")


## 6. Model-Free Prediction

Model-free methods learn directly from experience, without a model of the environment's dynamics. We will look at two model-free prediction methods: Monte Carlo and Temporal-Difference (TD) learning. These methods are used to estimate the value function of a policy.

### 6.1. Monte Carlo (MC) Prediction

Monte Carlo methods estimate the value function by averaging the returns observed after visiting a state. To use Monte Carlo methods, we need to have episodic tasks, meaning tasks that terminate.

The algorithm for first-visit MC prediction is as follows:

1.  Initialize `V(s)` and `Returns(s)` for all states `s`.
2.  Loop forever (for each episode):
    a. Generate an episode following the policy `π`.
    b. For each state `s` appearing in the episode:
        i.  `G` = return following the first occurrence of `s`.
        ii. Append `G` to `Returns(s)`.
        iii. `V(s)` = average of `Returns(s)`.

In [ ]:
def mc_prediction(env, policy, n_episodes, gamma=1.0):
    V = np.zeros(env.n_states)
    returns = {s: [] for s in range(env.n_states)}

    for _ in range(n_episodes):
        episode = []
        state = env.reset()
        done = False
        while not done:
            action = policy[state]
            next_state, reward, done, _ = env.step(action)
            episode.append((state, reward))
            state = next_state

        G = 0
        visited_states = set()
        for t in range(len(episode) - 1, -1, -1):
            state, reward = episode[t]
            G = gamma * G + reward
            if state not in visited_states:
                returns[state].append(G)
                V[state] = np.mean(returns[state])
                visited_states.add(state)
    return V

# Let's define a simple policy: move right if possible, otherwise move down.
policy = np.full(env.n_states, 2) # 2 is Right
policy[3] = 1 # Down
policy[7] = 1 # Down
policy[15] = 0 # Goal state, action doesn't matter

V_mc = mc_prediction(env, policy, n_episodes=1000)
print("Value function estimated by Monte Carlo:")
print(V_mc.reshape(env.grid_size, env.grid_size))

### 6.2. Temporal-Difference (TD) Prediction

Temporal-Difference (TD) learning is a combination of Monte Carlo and dynamic programming ideas. Like MC, TD methods learn directly from raw experience without a model of the environment's dynamics. Like DP, TD methods update estimates based in part on other learned estimates, without waiting for the final outcome (they bootstrap).

The TD(0) update rule is:

`V(S_t) <- V(S_t) + α[R_{t+1} + γV(S_{t+1}) - V(S_t)]`

where `α` is the learning rate.

In [ ]:
def td_prediction(env, policy, n_episodes, alpha=0.1, gamma=1.0):
    V = np.zeros(env.n_states)

    for _ in range(n_episodes):
        state = env.reset()
        done = False
        while not done:
            action = policy[state]
            next_state, reward, done, _ = env.step(action)
            td_target = reward + gamma * V[next_state]
            td_error = td_target - V[state]
            V[state] += alpha * td_error
            state = next_state
    return V

V_td = td_prediction(env, policy, n_episodes=1000)
print("Value function estimated by TD(0):")
print(V_td.reshape(env.grid_size, env.grid_size))

## 7. Model-Free Control

Control is about finding the optimal policy. In the model-free setting, we do not know the environment's dynamics. We will explore two popular TD control algorithms: Q-Learning (off-policy) and SARSA (on-policy).

### 7.1. Q-Learning (Off-Policy TD Control)

Q-Learning is an off-policy TD control algorithm. Off-policy means that the policy being learned about (the greedy policy) is different from the policy used to generate the data (the epsilon-greedy policy).

The Q-Learning update rule is:

`Q(S_t, A_t) <- Q(S_t, A_t) + α[R_{t+1} + γ * max_a Q(S_{t+1}, a) - Q(S_t, A_t)]`

We use an epsilon-greedy policy for action selection to ensure exploration.

In [ ]:
def q_learning(env, n_episodes, alpha=0.1, gamma=0.99, epsilon=0.1):
    Q = np.zeros((env.n_states, env.n_actions))

    for _ in range(n_episodes):
        state = env.reset()
        done = False
        while not done:
            if np.random.rand() < epsilon:
                action = np.random.randint(env.n_actions)
            else:
                action = np.argmax(Q[state])

            next_state, reward, done, _ = env.step(action)

            best_next_action = np.argmax(Q[next_state])
            td_target = reward + gamma * Q[next_state, best_next_action]
            td_error = td_target - Q[state, action]
            Q[state, action] += alpha * td_error

            state = next_state
    return Q

Q_q_learning = q_learning(env, n_episodes=10000)
policy_q = np.argmax(Q_q_learning, axis=1)

print("Optimal policy from Q-Learning:")
print(policy_q.reshape(env.grid_size, env.grid_size))

### 7.2. SARSA (On-Policy TD Control)

SARSA is an on-policy TD control algorithm. On-policy means that the policy being learned about is the same as the policy used to generate the data. The name SARSA comes from the sequence of events: State, Action, Reward, State, Action.

The SARSA update rule is:

`Q(S_t, A_t) <- Q(S_t, A_t) + α[R_{t+1} + γ * Q(S_{t+1}, A_{t+1}) - Q(S_t, A_t)]`

Notice the difference from Q-Learning: we use the Q-value of the action that was actually taken in the next state, not the maximum Q-value.

In [ ]:
def sarsa(env, n_episodes, alpha=0.1, gamma=0.99, epsilon=0.1):
    Q = np.zeros((env.n_states, env.n_actions))

    for _ in range(n_episodes):
        state = env.reset()
        done = False

        if np.random.rand() < epsilon:
            action = np.random.randint(env.n_actions)
        else:
            action = np.argmax(Q[state])

        while not done:
            next_state, reward, done, _ = env.step(action)

            if np.random.rand() < epsilon:
                next_action = np.random.randint(env.n_actions)
            else:
                next_action = np.argmax(Q[next_state])

            td_target = reward + gamma * Q[next_state, next_action]
            td_error = td_target - Q[state, action]
            Q[state, action] += alpha * td_error

            state = next_state
            action = next_action
    return Q

Q_sarsa = sarsa(env, n_episodes=10000)
policy_sarsa = np.argmax(Q_sarsa, axis=1)

print("Optimal policy from SARSA:")
print(policy_sarsa.reshape(env.grid_size, env.grid_size))

## 8. Deep Reinforcement Learning

For problems with large state spaces, using a table to store Q-values is not feasible. Deep Reinforcement Learning (DRL) uses deep neural networks to approximate the value function or the policy. This allows us to solve much more complex problems.

### 8.1. Deep Q-Networks (DQN)

A Deep Q-Network (DQN) is a neural network that approximates the action-value function `Q(s, a)`. The network takes the state `s` as input and outputs the Q-values for all possible actions.

Two key innovations in DQN are:

1.  **Experience Replay:** A buffer of past experiences `(s, a, r, s')` is stored. The network is trained on random mini-batches from this buffer, which breaks the correlation between consecutive samples.
2.  **Target Network:** A separate, fixed target network is used to generate the TD target. This helps to stabilize the training process.

In [ ]:
!pip install stable-baselines3[extra] gymnasium[box2d]

### 8.2. DQN with Stable Baselines3

We will use the `stable-baselines3` library to train a DQN agent on the `LunarLander-v2` environment. `stable-baselines3` is a set of reliable implementations of reinforcement learning algorithms in PyTorch.

In [ ]:
import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy

# Create the environment
env = gym.make('LunarLander-v2')

# Instantiate the DQN model
model = DQN('MlpPolicy', env, verbose=1)

# Train the model
model.learn(total_timesteps=100000)

# Evaluate the trained model
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

# Close the environment
env.close()

## 9. Problems and Solutions

This section contains a few exercises to help you solidify your understanding of the concepts covered in this notebook.

### Problem 1: Modify the GridWorld Environment

**Task:** Modify the `GridWorldEnv` to include a new element: a 'windy' condition. If the agent is in a certain column, there's a 50% chance it will be pushed one step up. 

**Hint:** Modify the `step` function. Add a condition to check if the agent is in a windy column and apply the wind effect with a certain probability.

In [ ]:
class WindyGridWorldEnv(GridWorldEnv):
    def __init__(self, grid_size=4, start_state=0, windy_cols=[1, 2]):
        super().__init__(grid_size, start_state)
        self.windy_cols = windy_cols

    def step(self, action):
        row, col = self._state_to_coords(self.agent_state)
        
        # Apply wind
        if col in self.windy_cols and np.random.rand() < 0.5:
            row = max(0, row - 1)

        if action == 0:  # Left
            col = max(0, col - 1)
        elif action == 1:  # Down
            row = min(self.grid_size - 1, row + 1)
        elif action == 2:  # Right
            col = min(self.grid_size - 1, col + 1)
        elif action == 3:  # Up
            row = max(0, row - 1)

        next_state = self._coords_to_state(row, col)
        self.agent_state = next_state

        if next_state == self.goal_state:
            reward = 1.0
            done = True
        elif next_state in self.holes:
            reward = -1.0
            done = True
        else:
            reward = 0.0
            done = False

        return next_state, reward, done, {}

# Test the windy environment
windy_env = WindyGridWorldEnv()
state = windy_env.reset()
windy_env.render()
print("\nTaking action in a windy column...")
windy_env.agent_state = 6 # (row=1, col=2), a windy column
next_state, reward, done, _ = windy_env.step(1) # Down
windy_env.render()
print(f"Next State: {next_state}, Reward: {reward}, Done: {done}")

### Problem 2: Implement Decaying Epsilon for Q-Learning

**Task:** Modify the `q_learning` function to include a decaying epsilon. The epsilon value should decrease over time, so the agent explores less as it learns more. A common strategy is to use exponential decay.

**Hint:** Add `epsilon_decay`, `min_epsilon` parameters. In each episode, update epsilon: `epsilon = max(min_epsilon, epsilon * epsilon_decay)`.

In [ ]:
def q_learning_decaying_epsilon(env, n_episodes, alpha=0.1, gamma=0.99, epsilon=1.0, min_epsilon=0.01, epsilon_decay=0.995):
    Q = np.zeros((env.n_states, env.n_actions))
    
    for episode in range(n_episodes):
        state = env.reset()
        done = False
        while not done:
            if np.random.rand() < epsilon:
                action = np.random.randint(env.n_actions)
            else:
                action = np.argmax(Q[state])

            next_state, reward, done, _ = env.step(action)

            best_next_action = np.argmax(Q[next_state])
            td_target = reward + gamma * Q[next_state, best_next_action]
            td_error = td_target - Q[state, action]
            Q[state, action] += alpha * td_error

            state = next_state
        
        epsilon = max(min_epsilon, epsilon * epsilon_decay)
    return Q

Q_q_learning_decay = q_learning_decaying_epsilon(env, n_episodes=10000)
policy_q_decay = np.argmax(Q_q_learning_decay, axis=1)

print("Optimal policy from Q-Learning with decaying epsilon:")
print(policy_q_decay.reshape(env.grid_size, env.grid_size))